In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
#from scipy.stats import mannwhitneyu
#import collections
#from statistics import mean
#from statistics import median
from collections import defaultdict
from Levenshtein import distance

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn import metrics
#from sklearn.metrics.cluster import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, silhouette_score

import copy

In [2]:
##############################
#
# The Dataset will only be available for the review
#
##############################


log = pd.read_csv("eventsWithPhases.csv")

In [3]:
#create log with Case ID based on currentQuestion + participant

#first change data type of currentQuestion from int to str
log = log.astype({'currentQuestion': str})
log.dtypes

#combine two columns
log['case_id'] = pd.factorize(log.participant+log.currentQuestion)[0]

#there are 614 cases, although 46 participants * 14 questions = 616 --> 2 cases are missing 
#print(len(log.case_id.unique()))

In [4]:
# Define conversion function
def convert_ms_to_date(milliseconds):
    date_obj = datetime.fromtimestamp(milliseconds / 1000.0)
    date_string = date_obj.strftime('%Y-%m-%d %H:%M:%S.%f')
    return date_string

# Apply conversion function to 'milliseconds' column
log['fixation_start'] = log['Fixation Start'].apply(convert_ms_to_date)
log['fixation_end'] = log['Fixation End'].apply(convert_ms_to_date)

In [5]:
# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'tabName_element' column
log['activity'] = le.fit_transform(log['tabName_element'])

#Number of unique activity values
#log['activity'].value_counts()

In [6]:
#Select only Control-flow questions
log_select = log[['case_id', 'fixation_start', 'activity', 'Phase', 'Type1', 'Type2', 'Type3', 'Fixation Duration']]
log_tasks = log_select.loc[log_select['Type2'] == 'Control-flow']

In [7]:
#log_tasks

In [8]:
#The experiment session with case_id 106 only contains 2 fixations and is therefore removed from the evaluation
#The error porbably occured since the participant involuntarily clicked on the 'next' button to go to the next task
log_tasks[log_tasks['case_id'] == 106]

,case_id,fixation_start,activity,Phase,Type1,Type2,Type3,Fixation Duration
31794,106,1970-01-01 01:38:40.855554,50,NaN,Local,Control-flow,Ordering,83.316
31795,106,1970-01-01 01:38:47.950454,412,NaN,Local,Control-flow,Ordering,66.624


In [9]:
log_tasks = log_tasks.drop(log_tasks[log_tasks['case_id'] == 106].index)
log_tasks

,case_id,fixation_start,activity,Phase,Type1,Type2,Type3,Fixation Duration
0,0,1970-01-01 02:35:15.800704,82,search,Local,Control-flow,Exclusiveness,83.4080
1,0,1970-01-01 02:35:16.708983,82,search,Local,Control-flow,Exclusiveness,124.9670
2,0,1970-01-01 02:35:16.883977,82,search,Local,Control-flow,Exclusiveness,66.6000
3,0,1970-01-01 02:35:17.900487,82,search,Local,Control-flow,Exclusiveness,83.3050
4,0,1970-01-01 02:35:18.375424,82,search,Local,Control-flow,Exclusiveness,108.3100
...,...,...,...,...,...,...,...,...
173748,610,1970-01-01 03:15:17.900596,88,NaN,Global,Control-flow,Ordering,191.6515
173749,610,1970-01-01 03:15:18.142225,179,NaN,Global,Control-flow,Ordering,283.3150
173750,610,1970-01-01 03:15:18.442194,179,NaN,Global,Control-flow,Ordering,141.6530
173751,610,1970-01-01 03:15:18.642171,50,NaN,Global,Control-flow,Ordering,191.6490


In [10]:
log_tasks['taskType'] = log_tasks['Type1'] + log_tasks['Type3']
log_group = log_tasks.groupby(['case_id'])['taskType'].apply(list).reset_index()
log_group['task'] = log_group['taskType'].apply(lambda x: str(set(x)))

In [11]:
#Create trace log
logVar = log_tasks.groupby(['case_id'])['activity'].apply(list).reset_index()
#len(logVar["activity"][310])
logVar

,case_id,activity
0,0,"[82, 82, 82, 82, 82, 177, 82, 82, 82, 82, 82, ..."
1,4,"[82, 82, 82, 181, 82, 82, 82, 82, 82, 82, 181,..."
2,5,"[90, 82, 183, 82, 82, 82, 82, 82, 183, 77, 75,..."
3,6,"[82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 8..."
4,7,"[86, 82, 82, 82, 82, 178, 178, 82, 82, 82, 82,..."
...,...,...
344,606,"[82, 59, 120, 122, 183, 120, 120, 183, 59, 183..."
345,607,"[82, 82, 82, 102, 175, 175, 175, 175, 175, 175..."
346,608,"[86, 86, 82, 108, 109, 86, 178, 86, 178, 108, ..."
347,609,"[82, 195, 195, 195, 195, 195, 195, 195, 195, 1..."


In [12]:
logVar["length"] = logVar["activity"].apply(lambda x: len(x))
#logVar.sort_values(by=['length'])
logVar['length'].max()

1152

In [13]:
logVar['length'].min()

27

In [14]:
#Create dictionary with true labels
log_group['label'] = le.fit_transform(log_group['task'])
labelDict1 = log_group['label'].to_dict()
labelDict1

{0: 5,
 1: 0,
 2: 3,
 3: 4,
 4: 7,
 5: 6,
 6: 2,
 7: 1,
 8: 5,
 9: 0,
 10: 3,
 11: 4,
 12: 7,
 13: 6,
 14: 2,
 15: 1,
 16: 5,
 17: 0,
 18: 3,
 19: 4,
 20: 7,
 21: 6,
 22: 2,
 23: 1,
 24: 5,
 25: 0,
 26: 3,
 27: 4,
 28: 7,
 29: 6,
 30: 2,
 31: 1,
 32: 5,
 33: 0,
 34: 3,
 35: 4,
 36: 7,
 37: 6,
 38: 2,
 39: 1,
 40: 5,
 41: 0,
 42: 3,
 43: 4,
 44: 7,
 45: 6,
 46: 2,
 47: 1,
 48: 5,
 49: 0,
 50: 3,
 51: 4,
 52: 7,
 53: 6,
 54: 2,
 55: 1,
 56: 5,
 57: 0,
 58: 3,
 59: 4,
 60: 7,
 61: 2,
 62: 1,
 63: 5,
 64: 0,
 65: 3,
 66: 4,
 67: 7,
 68: 6,
 69: 2,
 70: 1,
 71: 5,
 72: 0,
 73: 3,
 74: 4,
 75: 7,
 76: 6,
 77: 2,
 78: 1,
 79: 5,
 80: 0,
 81: 3,
 82: 4,
 83: 7,
 84: 6,
 85: 2,
 86: 1,
 87: 5,
 88: 0,
 89: 3,
 90: 4,
 91: 7,
 92: 6,
 93: 2,
 94: 1,
 95: 5,
 96: 0,
 97: 3,
 98: 4,
 99: 7,
 100: 6,
 101: 2,
 102: 1,
 103: 5,
 104: 0,
 105: 3,
 106: 4,
 107: 7,
 108: 6,
 109: 2,
 110: 1,
 111: 5,
 112: 0,
 113: 3,
 114: 4,
 115: 7,
 116: 6,
 117: 2,
 118: 1,
 119: 5,
 120: 0,
 121: 3,
 122: 4,
 12

In [15]:
trace_dis = log_group.groupby(['label'])['case_id'].apply(list).reset_index()
trace_dis['case_id'].str.len()


0    44
1    43
2    44
3    44
4    44
5    43
6    43
7    44
Name: case_id, dtype: int64

## Evaluation based on Nearest Neighbor

In [16]:
def NearestNeighbor(matrix, labelDict):
    
    clusterLabel = set(labelDict.values())
    clusterMeasuredCount = dict.fromkeys(list(clusterLabel), 0)
    
    for i in range(len(matrix)):
        #delete 0 in array matrix[i] for distance between (i,i)
        x = np.delete(matrix[i], i) 
        #identify min distance/value in array
        y = min(x)
        
        #identify position of y AND select first position/pair appearing in the array in case there are muliple pairs with identical min distance
        nearestNeighbor = np.where(x == y)[0] #problem if multiple positions??
        nearestNeighbor = int(nearestNeighbor[0])
        
        #true Label for i
        trueLabel = labelDict[i]
        
        #for label comparison add +1 to dict position if position occurs after i (because of the deletion of 0 at the beginning)
        if nearestNeighbor >= i:
            if trueLabel == labelDict[nearestNeighbor + 1]:
                clusterMeasuredCount[trueLabel] += 1         
        else:
            if trueLabel == labelDict[nearestNeighbor]:
                clusterMeasuredCount[trueLabel] += 1
    
    #Count the (true) number of traces per label/attribute
    clusterTrueCount = {}
    for i in clusterLabel:
        clusterTrueCount[i] = list(labelDict.values()).count(i)
    
    #Divide number of nearest neighbours with identical label by the respective (true number) of traces with this label
    metric = 0
    for i in clusterLabel:
        metric += clusterMeasuredCount[i] / clusterTrueCount[i]
        
    #print(clusterMeasuredCount, clusterTrueCount, metric)
    
    #print(clusterMeasuredCount)
    #print(clusterTrueCount)
    #print(clusterLabel)
    #print(metric)
    return metric / len(clusterLabel)


'''
x = np.array([[ 0, 12,  8, 10, 12],
       [5,  0,  9, 14, 17],
       [ 8,  9,  0, 12, 10],
       [ 8,  9,  1, 0, 10],
       [ 8,  9,  5, 4, 0]])

labelDict2 = {0:1,1:2,2:1,3:3,4:3}
NearestNeighbor(x, labelDict2)
'''

'\nx = np.array([[ 0, 12,  8, 10, 12],\n       [5,  0,  9, 14, 17],\n       [ 8,  9,  0, 12, 10],\n       [ 8,  9,  1, 0, 10],\n       [ 8,  9,  5, 4, 0]])\n\nlabelDict2 = {0:1,1:2,2:1,3:3,4:3}\nNearestNeighbor(x, labelDict2)\n'

## Evaluation based on Precision@k

In [17]:
def get_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    #return None  # Return None if the value is not found in the dictionary




def PrecisionAtK(matrix, labelDict, k):
    
    clusterLabel = set(labelDict.values())
    clusterMeasuredCount = dict.fromkeys(list(clusterLabel), 0)
    
    
    for i in range(len(matrix)):
        
        #true Label for i
        trueLabel = labelDict[i]

        
        #delete 0 in array matrix[i] for distance between (i,i)   
        x = np.delete(matrix[i], i)
        
        #get all k minimum values
        nnValues = []
        for l in range(k):
            y = min(x)
            nnValues.append(y)
            x = np.delete(x, np.where(x == y)[0][0])
        
        
        #get location of minimum values
        z = np.delete(matrix[i], i)
        
        #create dict from array with {position:value}
        my_dict = {}
        for m in range(len(z)):
            my_dict[m] = z[m]
        
        key_list = []
        for n in nnValues:
            key = get_key_by_value(my_dict, n)
            key_list.append(key)
            del my_dict[key]
        
        for o in range(k):
            position = list(key_list)[o]

            if position >= i:
                if trueLabel == labelDict[position + 1]:
                    clusterMeasuredCount[trueLabel] += 1

            else:
                if trueLabel == labelDict[position]:
                    clusterMeasuredCount[trueLabel] += 1

            #transform array to dict
            #get key value from dict
            #compare label
            #remove key+value from dict

            #Need exception in case y == 0 ??
            
    
    #Count the (true) number of traces per label/attribute
    clusterTrueCount = {}
    for i in clusterLabel:
        clusterTrueCount[i] = list(labelDict.values()).count(i)
    
    #Divide number of nearest neighbours with identical label by the respective (true number) of traces with this label
    metric = 0
    for i in clusterLabel:
        metric += clusterMeasuredCount[i] / k / clusterTrueCount[i]
        
    #print(clusterMeasuredCount, clusterTrueCount, metric)
    
    #print(clusterLabel)
    #print(metric)
    return metric / len(clusterLabel)


'''
x3 = np.array([[ 0, 12,  8, 10, 12],
       [5,  0,  9, 14, 17],
       [ 8,  9,  0, 12, 10],
       [ 8,  9,  1, 0, 10],
       [ 4,  8,  4, 4, 0]]) # --> Issue: How to select NN when identical distance values !!!

labelDict3 = {0:1,1:2,2:1,3:3,4:3}
PrecisionAtK(x3, labelDict3, 2)
'''

'\nx3 = np.array([[ 0, 12,  8, 10, 12],\n       [5,  0,  9, 14, 17],\n       [ 8,  9,  0, 12, 10],\n       [ 8,  9,  1, 0, 10],\n       [ 4,  8,  4, 4, 0]]) # --> Issue: How to select NN when identical distance values !!!\n\nlabelDict3 = {0:1,1:2,2:1,3:3,4:3}\nPrecisionAtK(x3, labelDict3, 2)\n'

## Triplet

see: https://towardsdatascience.com/triplet-loss-advanced-intro-49a07b7d8905

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
def get_triplet_mask(labels):
    
    # step 1 - get a mask for distinct indices
    ###print('labels', labels)
    # shape: (batch_size, batch_size)
    indices_equal = torch.eye(labels.size()[0], dtype=torch.bool, device=labels.device)
    ###print('equal', indices_equal)
    indices_not_equal = torch.logical_not(indices_equal)
    ###print('not_equal', indices_not_equal)

    # shape: (batch_size, batch_size, 1)
    i_not_equal_j = indices_not_equal.unsqueeze(2)
    ###print('i_not_j - unsqueeze2', i_not_equal_j)
    # shape: (batch_size, 1, batch_size)
    i_not_equal_k = indices_not_equal.unsqueeze(1)
    ###print('i_not_k - unsqueeze1', i_not_equal_k)
    # shape: (1, batch_size, batch_size)
    j_not_equal_k = indices_not_equal.unsqueeze(0)
    ###print('j_not_k - unsqueeze0', i_not_equal_k)
    # Shape: (batch_size, batch_size, batch_size)
    distinct_indices = torch.logical_and(torch.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
    ###print('distinct!!!!', distinct_indices)

    # step 2 - get a mask for valid anchor-positive-negative triplets
    # shape: (batch_size, batch_size)
    labels_equal = labels.unsqueeze(0) == labels.unsqueeze(1)
    ###print('labels_equal', labels_equal)
    # shape: (batch_size, batch_size, 1)
    i_equal_j = labels_equal.unsqueeze(2)
    ###print('i_equal_j', i_equal_j)
    # shape: (batch_size, 1, batch_size)
    i_equal_k = labels_equal.unsqueeze(1)
    ###print('i_equal_k', i_equal_k)
    # shape: (batch_size, batch_size, batch_size)
    valid_indices = torch.logical_and(i_equal_j, torch.logical_not(i_equal_k))
    ###print('valid_indices!!!', valid_indices)
    

    # step 3 - combine two masks
    mask = torch.logical_and(distinct_indices, valid_indices)
    ###print('mask!!', mask)
    return mask

    """compute a mask for valid triplets
    Args:
        labels: Batch of integer labels. shape: (batch_size,)
    Returns:
        Mask tensor to indicate which triplets are actually valid. Shape: (batch_size, batch_size, batch_size)
        A triplet is valid if:
        `labels[i] == labels[j] and labels[i] != labels[k]`
        and `i`, `j`, `k` are different.
    """
    
class custom_activation(nn.Module):
    def __init__(self):
        super(custom_activation, self).__init__()
    
    def forward(self, x):
        x[x>0] = 1
        x[x<=0] = 0
        return x


class BatchAllTtripletLoss(nn.Module):
  """Uses all valid triplets to compute Triplet loss

  Args:
    margin: Margin value in the Triplet Loss equation
  """
  def __init__(self, margin=0): #default margin = 0
    super().__init__()
    self.margin = margin
    self.relu = nn.ReLU() #new
    self.custom = custom_activation()
    
  def forward(self, distance_matrix, labels):
    """computes loss value.

    Args:
      embeddings: Batch of embeddings, e.g., output of the encoder. shape: (batch_size, embedding_dim)
      labels: Batch of integer labels associated with embeddings. shape: (batch_size,)

    Returns:
      Scalar loss value.
    """
    # step 1 - convert to tensor format
    distance_matrix = torch.tensor(distance_matrix)
    labels = torch.tensor(list(labels.values()))


    # step 2 - compute loss values for all triplets by applying broadcasting to distance matrix

    # shape: (batch_size, batch_size, 1)
    anchor_positive_dists = distance_matrix.unsqueeze(2)
    # shape: (batch_size, 1, batch_size)
    anchor_negative_dists = distance_matrix.unsqueeze(1)
    # get loss values for all possible n^3 triplets
    # shape: (batch_size, batch_size, batch_size)
    triplet_loss = anchor_negative_dists - anchor_positive_dists + self.margin
    ###print('tl0',triplet_loss)

    # step 3 - filter out invalid or easy triplets by setting their loss values to 0

    # shape: (batch_size, batch_size, batch_size)
    mask = get_triplet_mask(labels)
    ###print('mask', mask)
    triplet_loss *= mask
    ###print(triplet_loss)
    ###print('tl1:', triplet_loss)
    # easy triplets have negative loss values
    
    triplet_loss = self.custom(triplet_loss)
    ###print(triplet_loss)
    #triplet_loss = F.relu(triplet_loss)

    # step 4 - compute scalar loss value by averaging positive losses
    
    triLossNonZero = (triplet_loss != 0).nonzero(as_tuple=True)
    labelTorchUnique = torch.unique(labels, return_counts=True)
    
    nonZero = len(triLossNonZero[0])
    triLossSum = []
    for i in range(nonZero):
        #Identify L_a --> In Class
        t1 = triLossNonZero[0][i]
        labelIn = labels[t1]
        positionIn = int((labelTorchUnique[0] == labelIn).nonzero(as_tuple=False))
        countIn = labelTorchUnique[1][positionIn]

        #Identify L_b --> Out Class
        t3 = triLossNonZero[2][i]
        labelOut = labels[t3]
        positionOut = int((labelTorchUnique[0] == labelOut).nonzero(as_tuple=False))
        countOut = labelTorchUnique[1][positionOut]

        #Calculate loss
        value = (1/countIn)*(1/countIn)*(1/countOut)  
        ###print(countIn)
        ###print(countOut)
        triLossSum.append(value)
    
    #finally divide by |A|^2-|A|
    A = len(labelTorchUnique[0])  
    lossValue = sum(triLossSum) / (A*A-A)
        
    #OLD
    #E_triplet = (1 / (A^2 - A)) *
    #num_positive_losses = (triplet_loss > eps).float().sum()
    #print(num_positive_losses)
    #print(triplet_loss.sum())
    #triplet_loss = triplet_loss.sum() / (num_positive_losses + eps)
    

    return lossValue

## Silhouette

In [20]:
from sklearn import metrics

def Silhouette(distMatrix, labelDict):
    labelDictList = list(labelDict.values())
    return metrics.silhouette_score(distMatrix, labelDictList)

## Ground truth comparison

In [21]:
#Create dictionary with true labels
log_group['label'] = le.fit_transform(log_group['task'])
labelDict1 = log_group['label'].to_dict()
#labelDict1

In [22]:
logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(i) for i in x])
logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [23]:
def matrix_calc(features, distance):
    n = len(features)
    dist_matrix = np.zeros((n,n))
    
    for i in range(n):
        for j in range(i, n):
            dist_matrix[i,j] = distance(features[i], features[j])
            dist_matrix[j,i] = dist_matrix[i,j]
    
    return dist_matrix

In [24]:
def results(DistMatrix):
    print('NN:   ' + str(NearestNeighbor(DistMatrix, labelDict1)))
    print('P@10: ' + str(PrecisionAtK(DistMatrix, labelDict1, 10)))

    triplet = BatchAllTtripletLoss()
    print('Tri:  ' + str(triplet.forward(DistMatrix,labelDict1)))
    print('Sil:  ' + str(Silhouette(DistMatrix, labelDict1)))

### Levenshtein Distance

In [38]:
from Levenshtein import distance

List = list(logVar["strings"])

dist_matrix = np.zeros((len(List),len(List)),dtype=int)

for i in range(0,len(List)):
    for j in range(0,len(List)):
        dist_matrix[i,j] = distance(List[i],List[j])

lev_dis = dist_matrix
lev_dis

array([[  0, 267, 316, ..., 263, 274, 332],
       [267,   0, 311, ..., 188, 180, 311],
       [316, 311,   0, ..., 333, 307, 319],
       ...,
       [263, 188, 333, ...,   0, 194, 345],
       [274, 180, 307, ..., 194,   0, 351],
       [332, 311, 319, ..., 345, 351,   0]])

In [45]:
results(lev_dis)

NN:   0.8972647991543341
P@10: 0.7623678646934461
Tri:  tensor(0.6691)
Sil:  -0.22453133331372324


### Normalized Levenshtein Distance

In [40]:
List = logVar["strings"]

n = len(List)
dist_matrix = np.zeros((n,n))    # initialize distance matrix to a square of zeros

for i in range(n):
    for j in range(i, n):
        dist_matrix[i,j] = distance(List[i], List[j]) / max(len(List[i]),len(List[j]))
        dist_matrix[j,i] = dist_matrix[i,j]       # for the symmetric part, no computation

lev_dis_norm = dist_matrix

In [46]:
results(lev_dis_norm)

NN:   0.9371696617336153
P@10: 0.8652946617336154
Tri:  tensor(0.8727)
Sil:  0.10736466133082465


### Cosine based on 1-gram

In [23]:
#Create 1-gram

def createVector(charList):
    #dtype = [('structure', 'S10'), ('relfrequ', float)]
    arrayList = np.array(charList)
    unique, counts = np.unique(arrayList, return_counts=True)
    #calculate relative frequency
    relFrequList = np.array((unique, counts)).T
    uniqueList = list(unique)
    return relFrequList[relFrequList[:, 0].argsort()]
    #check completeness
    #if 'tree' not in uniqueList:
        #relFrequList = np.append(relFrequList, np.array([['tree', 0]]), axis=0)
        #print(relFrequList)

        
#Change data format from string to list of unique characters
logVar["1-gram"] = logVar["c:n_chr"].apply(lambda x: createVector(tuple(x)))
#logVar


In [24]:
def alignArrays(array1, array2):
    commonSet = set(array1[:,0]).union(array2[:,0])
    
    for i in commonSet:
        if i not in array1[:,0]:
            array1 = np.append(array1, np.array([[i, '0']]), axis=0)
        if i not in array2[:,0]:
            array2 = np.append(array2, np.array([[i, '0']]), axis=0)
    return array1[array1[:, 0].argsort()], array2[array2[:, 0].argsort()]

In [25]:
from scipy.spatial import distance

def cosineDist(frequVector1, frequVector2):
    Vector1, Vector2 = alignArrays(frequVector1, frequVector2)
    a = Vector1[:,1].astype(int)
    b = Vector2[:,1].astype(int)
    dist_matrix = distance.cosine(a, b)
    return dist_matrix

In [27]:
#Cosine distance based on 1-gram

cos1_dis = matrix_calc(logVar["1-gram"],cosineDist)

In [47]:
results(cos1_dis)

NN:   0.9772727272727273
P@10: 0.9241477272727272
Tri:  tensor(0.8901)
Sil:  0.22334463230599214


### Cosine based on 2-gram

In [30]:
#Change data format from string to list of unique characters
#logVar["charList"] = logVar["trace_variant"].apply(lambda x: list(x))
#logVar

def df_list(list_of_char):
    extList = list_of_char.copy()
    extList.insert(0, '*') 
    extList.append('$')
    list_new = []
    for i in range(len(extList)):
        new = ''.join(extList[i:i+2])
        list_new.append(new)
    del list_new[-1]
    return list_new

#Change data format from string to list of unique characters
logVar["dfList"] = logVar["c:n_chr"].apply(lambda x: df_list(x))
#logVar

logVar["2-gram"] = logVar["dfList"].apply(lambda x: createVector(x))
#logVar

In [48]:
#Cosine distance based on 2-gram

cos2_dis = matrix_calc(logVar["2-gram"],cosineDist)
results(cos2_dis)

NN:   0.9515063424947147
P@10: 0.8421643763213531
Tri:  tensor(0.8221)
Sil:  0.08707110224755636


In [34]:
#Create 3-gram

#Change data format from string to list of unique characters
logVar["charList"] = logVar["c:n_chr"].apply(lambda x: list(x))

def df_list2(list_of_char):
    extList = list_of_char.copy()
    extList.insert(0, '*') 
    extList.append('$')
    list_new = []
    for i in range(len(extList) - 1):
        new = ''.join(extList[i:i+3])
        list_new.append(new)
    del list_new[-1]
    return list_new


In [35]:
logVar["dfList2"] = logVar["charList"].apply(lambda x: df_list2(x))
logVar["3-gram"] = logVar["dfList2"].apply(lambda x: createVector(x))

In [49]:
#Cosine distance based on 3-gram

cos3_dis = matrix_calc(logVar["3-gram"],cosineDist)
results(cos3_dis)

NN:   0.8682610993657506
P@10: 0.7532835623678646
Tri:  tensor(0.7620)
Sil:  -0.0012724927871462736


In [50]:
#Aggregation

aggregate1 = cos1_dis + cos2_dis
results(aggregate1)

NN:   0.9715248414376322
P@10: 0.8948929704016915
Tri:  tensor(0.8620)
Sil:  0.1573427550043208


In [51]:
#Aggregation

aggregate2 = cos1_dis + cos2_dis + cos3_dis
results(aggregate2)

NN:   0.9543472515856237
P@10: 0.8604981501057083
Tri:  tensor(0.8380)
Sil:  0.10582398018892779


## Euclidean Distance

In [52]:
# Euclidean distance
# see https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy

def euclidDist(frequVector1, frequVector2):
    Vector1, Vector2 = alignArrays(frequVector1, frequVector2)
    a = Vector1[:,1].astype(float)
    b = Vector2[:,1].astype(float)
    euclidean_dist = np.linalg.norm(a-b)
    return euclidean_dist

In [53]:
#Euclidean distance based on 1-gram

euc1_dis = matrix_calc(logVar["1-gram"],euclidDist)
results(euc1_dis)

NN:   0.9542811839323467
P@10: 0.8622225158562368
Tri:  tensor(0.7647)
Sil:  -0.05617607680390533


In [54]:
#Euclidean distance based on 2-gram

euc2_dis = matrix_calc(logVar["2-gram"],euclidDist)
results(euc2_dis)

NN:   0.885438689217759
P@10: 0.7075184989429175
Tri:  tensor(0.6804)
Sil:  -0.12094105926059776


In [55]:
#Euclidean distance based on 3-gram

euc3_dis = matrix_calc(logVar["3-gram"],euclidDist)
results(euc3_dis)

NN:   0.6762024312896405
P@10: 0.5240420190274842
Tri:  tensor(0.6136)
Sil:  -0.14871501783104946


In [56]:
#Aggregation

agg_euc = euc1_dis + euc2_dis
results(agg_euc)

NN:   0.9427193446088795
P@10: 0.8232756342494715
Tri:  tensor(0.7365)
Sil:  -0.08381334058999264


### Jaccard based on 1-gram

In [57]:
#Distance based on activity type

def jaccard_similarity(list1, list2):
    s1, s2 = set(list1), set(list2)
    return 1 - len(s1 & s2) / len(s1 | s2)

In [58]:
#Jaccard based on 1-gram
Jacc1_dis = matrix_calc(logVar["charList"],jaccard_similarity)
results(Jacc1_dis)

NN:   0.9655787526427062
P@10: 0.9051268498942917
Tri:  tensor(0.8467)
Sil:  0.15975306042991522


In [59]:
#Jaccard based on 2-gram
Jacc2_dis = matrix_calc(logVar["dfList"],jaccard_similarity)
results(Jacc2_dis)

NN:   0.9827563424947146
P@10: 0.9583047040169133
Tri:  tensor(0.9151)
Sil:  0.11957225318299027


In [60]:
#Jaccard based on 3-gram
Jacc3_dis = matrix_calc(logVar["dfList2"],jaccard_similarity)
results(Jacc3_dis)

NN:   0.9857293868921776
P@10: 0.9660280126849894
Tri:  tensor(0.9193)
Sil:  0.05107219772286129


In [61]:
#Aggregation

agg_jacc1 = Jacc1_dis + Jacc2_dis
results(agg_jacc1)

NN:   0.9886363636363636
P@10: 0.9360002642706132
Tri:  tensor(0.8779)
Sil:  0.15678760609316028


In [62]:
agg_jacc2 = Jacc1_dis + Jacc2_dis + Jacc3_dis
results(agg_jacc2)

NN:   0.9885702959830867
P@10: 0.9520084566596195
Tri:  tensor(0.8912)
Sil:  0.1305832957211289


## Graph based measures

In [ ]:
#Now consider edge types

In [107]:
def intEncoder(character_List):
    return [np.where(np.array(list(dict.fromkeys(character_List)))==e)[0][0]for e in character_List]

logVar["intList"] = logVar["activity"].apply(lambda x: intEncoder(x))

In [108]:
# 2. transfer intList to int_tupleList

#Create tuple lists
def tuple_list(list_of_encodedActivities):
    #list.insert(0, '*')
    #list.append('*')
    list_new = []
    last_element = list_of_encodedActivities[-1]
    for i in range(len(list_of_encodedActivities)):
        new = tuple(list_of_encodedActivities[i:i+2])
        list_new.append(new)
    del list_new[-1]
    if list_of_encodedActivities.count(last_element) == 1: #check wether last activity in trace has some adjancency relation
        list_new.append((last_element,)) ### NOT Correct
    return list_new

#q = [0,0,0,0,1,1,2,3,4,5,3,2,4,0,5,6]
#tuple_list(q)

logVar["int_tupleList"] = logVar["intList"].apply(lambda x: tuple_list(x))

In [109]:
# 3. generate Adjacency List

def adj_list(list_of_tuples):
    adj_list_new = {}
    try:
        for node1, node2 in list_of_tuples:
            #print(node1, node2)
            if node1 not in adj_list_new:
                newlist = []
                newlist.append(node2)
                adj_list_new[node1] = newlist
                #print(adj_list3)
        
            else:
                if node2 not in adj_list_new[node1]:
                    #mylist.extend(adj_list3[node1])
                    adj_list_new[node1].append(node2)
                    #print(adj_list3)
                    #adj_list3[node1] = mylist
    
    #in case activity has no adjacent activity - only possible for last activity --> tuple: (lastAct,)
    except ValueError as ve:
        lastValue = list_of_tuples[-1][0] 
        adj_list_new[lastValue] = list()
    return list(adj_list_new.values())

#q = [0,0,0,0,1,1,2,3,4,5,3,2,4,0,5,6]
#l = tuple_list(q)
#adj_list(l)

logVar["int_adjList"] = logVar["int_tupleList"].apply(lambda x: adj_list(x))
#logVar["int_adjList"]

In [110]:
#Now consider length

from collections import deque

def bfs_4(graph, start, end):
    
    graph = {v: k for v, k in enumerate(graph)}
    #print(start, end)
    queue = deque([(start, 0)])
    seen = set()
    while queue:
        #print(queue)
        node, distance = queue.popleft()
        #if not node:
            #print(start, end, queue)
            #print("GRAPH LIST", graph)
        if node in seen:
            continue
        seen.add(node)
        if node == end:
            return distance 
        for adjacent in graph.get(node, []):
            queue.append((adjacent, distance + 1))
        
#x = {0: [0, 1], 1: [2, 1, 0, int], 2:[2], [3: [1, 5, 3, 7], 4: [3], 5: [6, 5], 6: [1, 7], 7: [8, 9, 7], 8: [5, 8, 10], 9: [3]}
#y = [[0, 1, 5], [1, 2], [3, 4], [4, 2], [5, 0], [3, 6], []]
#bfs_4(y, 1, 6)

In [111]:
from collections import defaultdict, deque
import copy

def reverse_graph(graph):
    reversed_graph = defaultdict(list)
    for node in graph:
        for neighbor in graph[node]:
            reversed_graph[neighbor].append(node)
    return reversed_graph


def bfs_5(graph, start, end):
    queue = deque([(start, 0)])
    seen = set()
    visited = {}
    while queue:
        node, distance = queue.popleft()
        if node in seen:
            continue
        seen.add(node)
        if node == end: # maybe quicker if adjacent directly checked
            return visited
        for adjacent in graph.get(node, []):
            queue.append((adjacent, distance + 1))
            if adjacent not in visited:
                visited.update({adjacent:distance})

            
def common_ancestors(graph, node1, node2): 
    #remove cross type edge between node1 and node2
    graph = copy.deepcopy(graph)
    graph[node1].remove(node2)
    graph = {v: k for v, k in enumerate(graph)}
    graphReverse = reverse_graph(graph)
    setNode1 = bfs_5(graphReverse, node1, 0)
    setNode2 = bfs_5(graphReverse, node2, 0)
    if next((a for a in list(setNode1) if a in list(setNode2)), None) == None:
        firstCommonAnces = next((a for a in list(setNode2) if a in list(setNode1)), None)
    else:
        firstCommonAnces = next((a for a in list(setNode1) if a in list(setNode2)), 0)
    
    #uses a hash map to identify the first common ancestor in both lists
    #looks for the first common ancestor in setNode1, which can also be found in setNode2 
    #--> this might not be the closest distance between setNode1 and setNode2
    #--> e.g., for x= [0,1,3,7,5,6] and y= [4,5,7,8,3] 7 might be closest ancestor, although algo detects 3 !
    #distance = setNode1[firstCommonAnces] + setNode2[firstCommonAnces]
    
    
    if firstCommonAnces != None:  
        ancesDistNode1 =  setNode1[firstCommonAnces] + 1 #the edge from node1 to first parent is counted as 0 by algorithm, therefore +1
        ancesDistNode2 =  setNode2[firstCommonAnces] + 1
        numberSkips = abs(ancesDistNode1 - ancesDistNode2)
        numberCross = min(ancesDistNode1, ancesDistNode2)
    else:
        numberSkips, numberCross = (0,1)
    return numberSkips, numberCross
    #if all(x in crossType for x in i):
    

    

#graphList = [[1], [2, 4, 1], [3, 2, 1], [], [5, 4], [5, 4, 6], [7], []]
#c = [[1, 4], [2], [3], [0, 5], [3, 5], []]
#c2 = {v: k for v, k in enumerate(c)}
#common_ancestors(c, 4, 5)
#reverse_graph(c2)

In [112]:
#Create List for decoding traces
from collections import OrderedDict
logVar["indexList"] = logVar["activity"].apply(lambda x: list(OrderedDict.fromkeys(x)))

### Cosine Edge Type + length (no df relations)

In [113]:
class Graph1:
    # instance variables
    def __init__(self, graph_list2, indexList):
        # v is the number of nodes/vertices
        self.time = 0
        self.traversal_array = []
        self.structural_array = [['sequ', 1]]
        #self.structural_array = []
        self.graph_list = graph_list2
        self.v = len(graph_list2)
        self.indexList = indexList

    # function for dfs
    def dfs(self):
        self.start_time = [-1]*self.v
        self.end_time = [-1]*self.v
 
        for node in range(self.v):
            if self.start_time[node] == -1:
                self.traverse_dfs(node)
                
        return np.array(self.structural_array)
        #print()
        #print("DFS Traversal: ", self.traversal_array)
        #print()
 
    def traverse_dfs(self, node):
        self.traversal_array.append(node)
        # get the starting time
        self.start_time[node] = self.time
        self.time += 1
        # traverse through the neighbours
        for neighbour in self.graph_list[node]:

            # when the neighbor was not yet visited
            if self.start_time[neighbour] == -1:                
                self.structural_array[0][1] += 0
                self.traverse_dfs(neighbour)
                
            # otherwise when the neighbour's visit is still ongoing:
            elif self.end_time[neighbour] == -1:
                if node == neighbour:
                    self.structural_array.append(['1back ',1])
                    #self.structural_array.append(['back ',1])
                    #self.structural_array.append([str(1)+'b'])
                
                elif node in self.graph_list[neighbour]:
                    self.structural_array.append(['2back ',2])
                    #self.structural_array.append(['back ',2])
                    #self.structural_array.append(str(2)+'b')
                    
                else:
                    x = bfs_4(self.graph_list, neighbour, node)
                    self.structural_array.append([str(x+1)+'back ',x+1])
                    #self.structural_array.append(['back ',x+1])
                    #self.structural_array.append(str(x+1)+'b')
                
            # otherwise when the neighbour's visit started before the current node's visit:
            elif self.start_time[node] < self.start_time[neighbour]:
                graph_list_copy = copy.deepcopy(self.graph_list)
                graph_list_copy[node].remove(neighbour)
                y = bfs_4(graph_list_copy, node, neighbour)
                self.structural_array.append([str(y-1)+'forward ',y-1])

            else:
                numberSkips, numberCross = common_ancestors(self.graph_list, node, neighbour)
                self.structural_array.append([str(numberCross)+'cross ',numberCross])
  
    
        # Indentation corrected:
        self.end_time[node] = self.time
        self.time += 1

In [115]:
from collections import Counter

def transform_list_of_pairs(pairs):
    return [pair[0] for pair in pairs]



def count_entries(input_list):
    # Count the occurrences of each unique entry in the list
    counter = Counter(input_list)
    
    # Create a NumPy array from the counter dictionary
    result = np.array([[key, count] for key, count in counter.items()], dtype=object)
    
    return result

# Example usage
#input_list = ['sequ', '2back', '2back']
#result = count_entries(input_list)
#print(result)

logVar["int_strucLengthList2"] = logVar.apply(lambda x: Graph1(x.int_adjList, x.indexList).dfs(), axis =1)
logVar["relFrequVec1"] = logVar["int_strucLengthList2"].apply(lambda x: transform_list_of_pairs(x))
logVar["relFrequVec1"] = logVar["relFrequVec1"].apply(lambda x: count_entries(x))

In [117]:
#Cosine distance based on edge types
cos_graph_dis = matrix_calc(logVar["relFrequVec1"],cosineDist)
results(cos_graph_dis)

NN:   0.17197410147991543
P@10: 0.15701638477801266
Tri:  tensor(0.5332)
Sil:  -0.13404679651077211


In [118]:
#Cosine distance based on edge types
agg_cos = cos1_dis + cos2_dis + cos_graph_dis
results(agg_cos)

NN:   0.9571220930232558
P@10: 0.898090644820296
Tri:  tensor(0.8625)
Sil:  0.15896867249184918


### Jaccard Edge Type and length + df relation

In [119]:
class Graph2:
    # instance variables
    def __init__(self, graph_list2, indexList):
        # v is the number of nodes/vertices
        self.time = 0
        self.traversal_array = []
        self.structural_array = []
        #self.structural_array = []
        self.graph_list = graph_list2
        self.v = len(graph_list2)
        self.indexList = indexList

    # function for dfs
    def dfs(self):
        self.start_time = [-1]*self.v
        self.end_time = [-1]*self.v
 
        for node in range(self.v):
            if self.start_time[node] == -1:
                self.traverse_dfs(node)
                
        return self.structural_array
        #print()
        #print("DFS Traversal: ", self.traversal_array)
        #print()
 
    def traverse_dfs(self, node):
        self.traversal_array.append(node)
        # get the starting time
        self.start_time[node] = self.time
        self.time += 1
        # traverse through the neighbours
        for neighbour in self.graph_list[node]:

            # when the neighbor was not yet visited
            if self.start_time[neighbour] == -1:                
                #self.structural_array[0][1] += 0
                #self.structural_array.append('tree ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ')
                self.structural_array.append('tree')
                self.traverse_dfs(neighbour)
                
            # otherwise when the neighbour's visit is still ongoing:
            elif self.end_time[neighbour] == -1:
                if node == neighbour:
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(1))
                    #self.structural_array.append([str(1)+'b'])
                
                elif node in self.graph_list[neighbour]:
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(2))
                    #self.structural_array.append(str(2)+'b')
                    
                else:
                    x = bfs_4(self.graph_list, neighbour, node)
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(x+1))
                    #self.structural_array.append(str(x+1)+'b')
                
            # otherwise when the neighbour's visit started before the current node's visit:
            elif self.start_time[node] < self.start_time[neighbour]:
                graph_list_copy = copy.deepcopy(self.graph_list)
                graph_list_copy[node].remove(neighbour)
                y = bfs_4(graph_list_copy, node, neighbour)
                self.structural_array.append('forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(y-1))
                #self.structural_array.extend((y-1)*['forward']) # -1 to exclude one edge: (A:B,C;B:C;C:[]) ...the dist A --> C is 2 without forward edge, but we are skipping only one activity
                #self.structural_array.append(str(y-1)+'f')
                
            else:
                #Possibly first check, whether two nodes connected by cross-type have identical parent
                numberSkips, numberCross = common_ancestors(self.graph_list, node, neighbour)
                #self.structural_array.append('forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(numberSkips))
                self.structural_array.append('cross ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(numberCross))
                #self.structural_array.append(str(numberSkips)+'f')
                #self.structural_array.append(str(numberCross)+'c')
    
        # Indentation corrected:
        self.end_time[node] = self.time
        self.time += 1


In [120]:
logVar["int_strucLengthList3"] = logVar.apply(lambda x: Graph2(x.int_adjList, x.indexList).dfs(), axis =1)
#logVar

In [121]:
#Jacc similarity based on edge types

jacc_graph = matrix_calc(logVar["int_strucLengthList3"],jaccard_similarity)
results(jacc_graph)

NN:   0.960029069767442
P@10: 0.9242798625792813
Tri:  tensor(0.9026)
Sil:  0.08512902120313036


In [122]:
#Jacc sim based on edge types
agg_jacc = Jacc1_dis + Jacc2_dis + jacc_graph
results(agg_cos)

NN:   0.9571220930232558
P@10: 0.898090644820296
Tri:  tensor(0.8625)
Sil:  0.15896867249184918


## Eventually Follows

In [63]:
#Spatial distance between strings


from scipy.spatial import distance


def distanceSpatial(traceString, char1, char2):
    positions_letter1 = [pos for pos, char in enumerate(traceString) if char == char1]
    positions_letter2 = [pos for pos, char in enumerate(traceString) if char == char2]
    
    distList = []
    

    for i in range(len(positions_letter1)):
        for j in range(len(positions_letter2)):
            dist = positions_letter2[j] - positions_letter1[i]
            if dist > 0:
                    #print(dist)
                distList.append(dist)
                    
    
    if not distList: #distList.append(0) #in the case the char1 is after char2 asign dist 0, i.e. char2 cannot be reached from char1
        return 0
    else:
        return 1/min(distList)





def commonDistance(trace1, trace2):
    
    commonSet = set(trace1) & set(trace2)

    commonList = list(commonSet)
    commonList.sort()
    #print(commonList)

    n = len(commonSet)
    dist_matrix1 = np.zeros((n,n))
    dist_matrix2 = np.zeros((n,n))

    for i in range(n):
        for j in range(i, n):
            dist_matrix1[i,j] = distanceSpatial(trace1, commonList[i], commonList[j])
        
    for i in range(n):
        for j in range(i, n):
            dist_matrix2[i,j] = distanceSpatial(trace2, commonList[i], commonList[j])
    
    #print(dist_matrix1, dist_matrix2)
    return distance.cosine(dist_matrix1.ravel(), dist_matrix2.ravel())



#x = 'ABCDEF'
#y = 'ABCDEBCDEBCDEF'
#z = 'ABCDEBCDEF'
#print(dist_matrix)
#distanceSpatial(x, 'A', 'E')
#listVec = logVar["strings"]
#x= listVec[0]
#y= listVec[1]
#commonDistance(x, y)

In [65]:
dist_matrix_evFollows = matrix_calc(logVar["strings"],commonDistance)
agg_evFollows = 0.7*dist_matrix_evFollows + 0.3*cos1_dis 
results(agg_evFollows)

NN:   0.46128435517970406
P@10: 0.4549550739957716
Tri:  tensor(0.8286)
Sil:  0.06766086823284875


## Maximal Repeat

In [68]:
from suffix_tree import Tree

In [69]:
#tree.maximal_repeats
def maxRepeat(tree):
    mrList=[]
    for C, path in sorted(tree.maximal_repeats()):
        mrList.append(str(path))
    return mrList

#test_tree = Tree({"A": "aaacdcdcbedbccbadbdebdc"})
#maxRepeat(test_tree)

['a', 'a a', 'b', 'b d', 'c', 'c b', 'c d c', 'd', 'd b', 'd c', 'e']

In [70]:
#create vector based on maximal repeats
logVar["mrList"] = logVar["strings"].apply(lambda x: maxRepeat(Tree({"A": x})))
logVar["mrVector"] = logVar["mrList"].apply(lambda x: createVector(tuple(x)))

In [71]:
#Cosine distance based on maxR

cos_mr = matrix_calc(logVar["mrVector"],cosineDist)
results(cos_mr)

NN:   0.9571220930232558
P@10: 0.9028409090909092
Tri:  tensor(0.8912)
Sil:  0.18629615358356524


In [72]:
#Euclidean distance based on maxR

euc_mr = matrix_calc(logVar["mrVector"],euclidDist)
results(euc_mr)

NN:   0.7854122621564482
P@10: 0.6207650634249472
Tri:  tensor(0.6617)
Sil:  -0.06668386684998291


In [73]:
#Jaccard similarity based on maxR

jacc_mr = matrix_calc(logVar["mrList"],jaccard_similarity)
results(jacc_mr)


NN:   0.9541490486257929
P@10: 0.9013940274841438
Tri:  tensor(0.8847)
Sil:  0.1362689156049775


# Trace2Vec

In [26]:
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity

logVar["words"] = logVar["c:n_chr"].apply(lambda x: ' '.join(x))
logVar["words"][0]

'R R R R R ± R R R R R R ± R ± R R R R ± ± ± ± ± ± R R R R R R R R R R R R R 4 R 2 2 2 O Q O H M J L ė ė ũ Ɯ Ɯ Ŭ ƙ Ů Ʊ Æ Æ Ë Ë É É Ă Ă Ă Ă Ă È Ę Ę Ƴ ƴ Ƹ Ũ Ũ Ů Ů Ů Ů Ů Ū Ū Ɯ ȹ ȁ ȁ ǻ ǿ ǿ ǿ ǿ Ȼ Ǿ Ǿ Ȼ ȃ ǽ ǽ Ȁ Ȁ ȫ Ȣ ɐ ɖ ˦ ˒ ʢ ʞ ʞ ʞ ʞ ʞ ʞ ʝ ʝ ʠ ʠ ˒ ˨ ˦ ˩ ͖ ̢ ̦ ̦ ͖ ͉ ͉ ͖ ͘ Ω Ͳ Ͳ Ͳ Ϭ Ϩ θ ϗ ϗ ϗ ϖ ϗ φ Ы Ы Ы а ѥ і з Ϭ ϩ χ φ π ϔ ͖ ͖ ͖ ̷ ̧ ̧ ̧ ̤ ͖ ͖ ̢ ̢ ͖ ̣ ̣ ̣ ̣ Ͳ ͯ Ͱ ͯ ͮ ͮ ͯ ͭ ͯ ͫ ͪ Ͳ ͩ ͨ ͬ ˤ ˢ ˒ ˋ ˒ ʢ ʝ ˬ ˬ ˬ ˬ ˒ ʟ ʟ ˒ ʡ ˒ ʠ ʠ ʠ ʞ ʢ ʞ ˨ ˨ Ǽ Ȍ Ȼ ȱ Ȼ Ȼ Ȼ ȁ ȁ ɓ ɒ ɣ ʐ ʐ ɥ ʐ ɖ ɖ ɖ Ƚ ɖ ʐ ɖ Ȁ Ȁ Ȁ Ȁ Ȁ ȹ ȹ Ƚ Ƚ ɘ ɘ Ȼ ǻ ȹ Ȱ Ȼ Ȼ ȅ ȏ ȁ ʐ ɖ ɥ ɖ Ȁ Ȁ ȯ ȯ Ⱥ Ⱥ Ȼ Ȼ Ȼ Ȼ Ⱥ ɘ ȹ Ȼ Ȁ Ȼ ȁ Ȼ ȁ ȁ ȁ Ȼ ɘ ɘ'

In [27]:
documents = list(logVar['words'])

# Step 1: Tokenize documents
texts = [
    [word for word in document.lower().split()]
    for document in documents
]

# Step 2: Tag the documents

from gensim.models.doc2vec import TaggedDocument

tagged_docs = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(documents)]

In [28]:
#Derive and evaluate model

from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity

def trainModel(documents,vec_size):
    
    # Step 3: Initialize and train the model
    model = Doc2Vec(vector_size=vec_size, window=3, min_count=0.0, min_alpha=0.025, epochs=10, dm = 0, alpha=0.025)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
    
    # Step 4: infer vectors for each document
    vectors = [model.infer_vector(doc) for doc in texts]
    
    # Step 5: calculate cosine distances
    cos = cosine_similarity(vectors)
    
    return results(cos)

In [29]:
trainModel(tagged_docs,16)

NN:   0.0
P@10: 0.0
Tri:  tensor(0.0564)
Sil:  0.24622111


In [30]:
trainModel(tagged_docs,32)

NN:   0.0
P@10: 0.0
Tri:  tensor(0.0653)
Sil:  0.23205745


In [31]:
trainModel(tagged_docs,64)

NN:   0.0
P@10: 0.0005681818181818183
Tri:  tensor(0.0674)
Sil:  0.21483004


In [32]:
trainModel(tagged_docs,128)

NN:   0.0
P@10: 0.0005747885835095138
Tri:  tensor(0.0710)
Sil:  0.21265528
